# Tarefa 4

Alunos:

* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("dados4.csv")
X = df.drop(['V15'], axis=1)
y = df.filter(['V15'], axis=1)
y.head()

,V15
0,0
1,0
2,0
3,1
4,1


In [3]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
0,1,22.08,11.46,2,k,bb,1.585,0,0,0,1,g,100,1213
1,0,22.67,7.00,2,c,bb,0.165,0,0,0,0,g,160,1
2,0,29.58,1.75,1,k,bb,1.250,0,0,0,1,g,280,1
3,0,21.67,11.50,1,j,j,0.000,1,1,11,1,g,0,1
4,1,20.17,8.17,2,aa,bb,1.960,1,1,14,0,g,60,159


## Preprocessamento

In [4]:
X = pd.get_dummies(X, columns=['V5', 'V6', 'V12'])
X.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,...,0,0,0,0,0,0,0,1,0,0
1,0,22.67,7.00,2,0.165,0,0,0,0,160,...,0,0,0,0,0,0,0,1,0,0
2,0,29.58,1.75,1,1.250,0,0,0,1,280,...,0,0,0,0,0,0,0,1,0,0
3,0,21.67,11.50,1,0.000,1,1,11,1,0,...,0,0,0,1,0,0,0,1,0,0
4,1,20.17,8.17,2,1.960,1,1,14,0,60,...,0,0,0,0,0,0,0,1,0,0


In [5]:
X = StandardScaler().fit_transform(X)

## Métodos

In [6]:
methods = {
    'Logistic_Regression': LogisticRegression(penalty='none'),
    'Logistic_Regression_l2': LogisticRegression(penalty='l2'),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'SVM_Linear': LinearSVC(),
    'SVM_RBF': SVC(kernel='rbf'),
    'Naive_Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest': RandomForestClassifier(),
    'GBM': GradientBoostingClassifier(),
}

params={
    'Logistic_Regression': {},
    'Logistic_Regression_l2': {'C': 10**(np.random.uniform(-3, 3, 10))},
    'LDA': {},
    'QDA': {},
    'SVM_Linear': {'C': 2**(np.random.uniform(-5, 15, 10))},
    'SVM_RBF': {'C': 2**(np.random.uniform(-5, 15, 10)),
                'gamma': 2**(np.random.uniform(-9, 3, 10))},
    'Naive_Bayes': {},
    'KNN': {'n_neighbors': np.random.randint(1, 301, 10)},
    'MLP': {'hidden_layer_sizes': np.arange(5, 21, 3)},
    'Decision_Tree': {'ccp_alpha': np.random.uniform(0.04, 0.0, 10)},
    'Random_Forest': {'n_estimators': [10, 100, 1000],
                      'max_features': [5, 8, 10]},
    'GBM': {'n_estimators':np.random.randint(5, 100, 10),
            'learning_rate': np.random.uniform(0.01, 0.3, 10),
            'max_depth': [2, 3]}
}

## Iterações

In [7]:
results = {
    'clf': [],
    'mean_auc_outer': []
}

for method in methods:
    print(method)
    auc_outer = []
    for iteration in range(4):
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            np.ravel(y),
            test_size=0.3)
        clf = GridSearchCV(
            methods[method],
            params[method],
            cv=StratifiedKFold(n_splits=3),
            scoring='roc_auc',
            refit=True)
        clf.fit(X_train, y_train)
        # get prediction on test (outer loop)
        y_pred = clf.predict(X_test)
        # calculate auc and save
        auc = roc_auc_score(y_test, y_pred)
        auc_outer.append(auc)
    mean_auc = np.mean(auc_outer)
    results['clf'].append(method)
    results['mean_auc_outer'].append(mean_auc)


Logistic_Regression
Logistic_Regression_l2
LDA
QDA
SVM_Linear
SVM_RBF
Naive_Bayes
KNN
MLP
Decision_Tree
Random_Forest
GBM


In [8]:
results_df = pd.DataFrame(results)
results_df

,clf,mean_auc_outer
0,Logistic_Regression,0.841158
1,Logistic_Regression_l2,0.859003
2,LDA,0.865399
3,QDA,0.686390
4,SVM_Linear,0.874589
5,SVM_RBF,0.857130
6,Naive_Bayes,0.714750
7,KNN,0.801193
8,MLP,0.833829
9,Decision_Tree,0.836149
